# Integrated Gradients: An interpretation method for deep learning models

Here is a brief overview of Integrated Gradients, a method we use to interpret our CNN model predictions. The main motivation behind Integrated Gradients is that when we use a vanilla gradient method to attribute feature importance, we run into the problem of *saturation*. Gradient saturation occurs when the model output stops changing even though the input is still important. For example, when the model has "confident" activations, the gradient can plateau, and the values will be close to zero, however the feature may still be important. Integrated Gradients solves this problem by considering the path from a baseline input (e.g. all zeros) to the actual input, and integrating the gradients along this path. Intuitively, we can think of this as starting at an extremely blurry or grey image, and at each step the image sharpens until we see the full resolution input. This way, we capture the total change in the output as we move from the baseline to the actual input, providing a more accurate attribution of feature importance. Another problem that vanilla gradient methods face is that they can be highly sensitive to noise: a small perturbation in the input can lead to large changes in the gradient, making the attributions unstable. Since Integrated Gradients considers the entire path from the baseline to the input, it averages out these small perturbations, leading to more stable and reliable attributions. This method was introduced by Sundararajan et al. [1] and has been widely adopted for interpreting deep learning models.

## Integrated Gradients Method (Sundararajan et al., 2017)

Given a model $ F: \mathbb{R}^n \to [0, 1] $ and an input $x \in \mathbb{R}^n $, the Integrated Gradients for the $i^{th} $ feature is defined as:

$$IG_i(x) = (x_i - x'_i) \times \int_{\alpha=0}^{1} \frac{\partial F(x' + \alpha \times (x - x'))}{\partial x_i} d\alpha$$

where $ x' $ is the baseline input (e.g., all zeros), and $ \alpha $ is a scaling factor that varies from 0 to 1. 

In practice this integral is approximated using a Riemann sum:

$$IG_i(x) \approx (x_i - x'_i) \times \frac{1}{m} \sum_{k=1}^{m} \frac{\partial F(x' + \frac{k}{m} \times (x - x'))}{\partial x_i}$$

where $ m $ is the number of steps in the approximation. A larger $ m $ leads to a more accurate approximation of the integral. Generally $m$ is chosen between 20 and 300, in our case we use $m=50$ due to computational constraints.

### Properties of Integrated Gradients

In the original paper by Sundararajan et al. (2017), the authors prove that Integrated Gradients satisfy two important properties:

1. **Sensitivity**: If the input differs from the baseline in only one feature and the model's output changes, then the attribution for that feature should be non-zero. This ensures that important features are correctly identified.

2. **Implementation Invariance**: If two models are functionally equivalent (i.e., they produce the same output for all inputs), then their attributions should also be identical. This property ensures that the attribution method is robust to different implementations of the same function.

Intuitively, Integrated Gradients can be thought of as walking from the baseline input to the full input, and accumulating the gradients along the way. This captures the total contribution of each feature to the change in the model's output, providing a more comprehensive and reliable attribution compared to vanilla gradient methods, whilst avoiding saturation and noise issues.

## SG - IG: SmoothGrad Integrated Gradients 

One way we can try to improve the stability of Integrated Gradients is to combine it with SmoothGrad, resulting in the SmoothGrad Integrated Gradients (SG-IG) method. The idea behind SG-IG is to smooth the function first using SmoothGrad, and them apply Integrated Gradients to the smoothed function. This helps to reduce noise and improve the stability of the attributions.

If we define SmoothGrad as:

$$SG(x) = \frac{1}{N} \sum_{i=1}^{N} \nabla F(x + \epsilon_i))$$

where $ N $ is the number of noisy samples, and $\epsilon_i \sim \mathcal{N}(0, \sigma^2) $ is Gaussian noise with mean 0 and variance $ \sigma^2 $, then the SG-IG for the $ i^{th} $ feature is defined as:

$$SG\text{-}IG_i(x) = \frac{1}{N} \sum_{j=1}^{N} IG_i(x + \epsilon_j)$$

where $ IG_i(x + \epsilon_j) $ is the Integrated Gradients for the noisy input $ x + \epsilon_j $.



# Intuition of interpolation between baseline and input
Here is a simple illustration of the interpolation process between a baseline image (all zeros) and an actual input image using Integrated Gradients. The images below show the gradual transition from the baseline to the input, and for each step, we compute the gradient of the model's output with respect to the input. By integrating these gradients along the path, we can attribute the importance of each pixel in the input image to the model's prediction.

This image was taken from the medium article [3] which is an excellent explainer for Integrated Gradients.


![IG path](image1.png)

## References

[1] Sundararajan, M., Taly, A., & Yan, Q. (2017). Axiomatic attribution for deep networks. In Proceedings of the 34th International Conference on Machine Learning-Volume 70 (pp. 3319-3328). JMLR. org.
\\
[2] Smilkov, D., Thorat, N., Kim, B., Viégas, F., & Wattenberg, M. (2017). Smoothgrad: removing noise by adding noise. arXiv preprint arXiv:1706.03825.

[3] Piro, K. (2020). XAI Methods: Integrated Gradients. Medium. https://medium.com/@kemalpiro/xai-methods-integrated-gradients-6ee1fe4120d8